## This code helps to get rid of many articles that are irrelevant to technology clusters

#### At this stage, we are only relying on the information available from title, text and and type of the articles

In [ ]:
import pandas as pd
import os 
import re
import numpy as np
import spacy
pd.set_option('mode.chained_assignment', None)

#Package that has libraries of various geolocations
from geotext import GeoText

In [ ]:
cat = pd.read_csv('enwiki-20191001-category.csv') #list if Wiki categories and subcategories 
page = pd.read_csv('enwiki-20191001-page.csv') #list of all pages with characterizing information
link= pd.read_csv('enwiki-20191001-categorylinks.csv') #links between categories and subcategories in Wiki

In [ ]:
page = pd.read_csv('enwiki-20191001-page.csv')
page=page[['id','namespace','title']]

#Leave only pages related to main articles and categories 
page=page[(page['namespace']==0) | (page['namespace']==14)] 
page=page.drop_duplicates()
page=page.reset_index(drop=True)

#Get rid of technical pages and clean title
stop_page='companion|user|templates|shared ip addresses from educational institutions|categories|category|composition|use mdy dates|use dmy dates|self-published work|cs1|self-published work|images of|lists of|redirects|redirected|userbox|webarchive|infobox|articles|wikipedia|pages|wikidata|files with no machine-readable|non-free|all free|wikimedia|all-free|webarchive template wayback links|wiki|knight|town|village|culture|history|bishop|school|street|student|restaurant|mayor|festival|death|politician|politics|architecture|musical group|musical band|writer|musician|tourist attraction|commander|king|queen|church|museum|university|gallery|cathedral|neighbourhood|companies|company|country|birth|death|missing|year|award|template:|infobox:|fictional|movie|theatre|tournament|styles|names|novel|people|album|artist|ballet|discography|dance|opera|contest|song|singer|actor|actress|performer|journal|magazine|newsapaper|television series|television episode|television channel|television program|bibliograph|genre|poem|country|municipality|village|historic|event|holiday|person|player|biography|religion|religious|company|military conflict|currency|celebrity'
page['title'] = page['title'].str.replace('_',' ')
page = page[~page['title'].str.contains(stop_page, na=False)]
page=page[~page['title'].str.contains(r'[0-9]', na=False)]
titles=page[['title']]
titles=titles.drop_duplicates()
titles=titles['title'].tolist()
titles=' '.join(str(v) for v in titles)



In [ ]:
#Get rid of pages related to geographic locations
places = GeoText(titles)
locations=places.cities+places.countries
locations = list(dict.fromkeys(locations))
locations_df=pd.DataFrame(locations, columns=['location'])
locations='|'.join(locations_df['location'].tolist())
page = page[~page['title'].str.contains(locations, na=False)]




####### Repeat the same thing for categories file and link file

In [ ]:
#Categories
cat['title'] = cat['title'].str.replace('_',' ')
cat = cat[~cat["title"].str.contains(locations, na=False)]
cat["title"] = cat["title"].str.lower()
cat = cat[~cat["title"].str.contains(stop, na=False)]
cat=cat.drop(['id'], axis=1)
cat=cat[~cat['title'].str.contains(r'[0-9]', na=False)]
cat.is_copy = None
cat=cat.drop(['files','subcats'], axis=1)
cat=cat.drop_duplicates()


#Links
link=link.drop(['sortkey','timestamp','collation'], axis=1)
link=link.dropna(subset=['type'])
link = link[link["type"].str.contains('page|subcat')]
import numpy as np
link = link.replace(np.nan, '', regex=True)
link['to'] = link['to'].str.replace('_',' ')
link = link[~link["to"].str.contains(locations, na=False)]
link["to"] = link["to"].str.lower()
link = link[~link["to"].str.contains(stop, na=False)]
link=link[~link['to'].str.contains(r'[0-9]', na=False)]

link['sortkey_prefix'] = link['sortkey_prefix'].str.replace('_',' ')
link["sortkey_prefix"]=link["sortkey_prefix"].str.lower()
link = link[~link["sortkey_prefix"].str.contains(stop, na=False)]
link = link[~link["sortkey_prefix"].str.contains(locations, na=False)]
link=link[~link['sortkey_prefix'].str.contains(r'[0-9]', na=False)]

In [ ]:
#Leave only connections related to links between pages and subcategories/categories
link=link.drop(['sortkey','timestamp','collation'], axis=1)
link=link.dropna(subset=['type'])
link = link[link["type"].str.contains('page|subcat')]
link = link.replace(np.nan, '', regex=True)